# LendingClub Predictions

Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. You can read more about their marketplace [here](https://www.lendingclub.com/public/how-peer-lending-works.action).

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. 
- Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. 
- The interest rate is the percent in addition to the requested loan amount the borrower has to pay back. 
- A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. 
- Each borrower is given a grade according to the interest rate they were assigned. 
- If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.
- Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application.
- The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Many loans aren't completely paid off on time, however, and some borrowers [default](https://www.lendingclub.com/investing/investor-education/collection-of-monthly-payments) on the loan.
- Lending Club releases data for all of the approved and declined loan applications periodically on their [website](https://www.lendingclub.com/info/download-data.action).
- The data dictionary for the data is present [here](https://github.com/ajdatahub/ProjectDS/tree/master/LendingClub%20Predictions). The LoanStats sheet describes the approved loans datasets and the RejectStats describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only.


- The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

    __Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?__


Before we can start doing machine learning, we need to clean our data and define what features we want to use and which column repesents the target column we want to predict.

### Data Cleaning

In [52]:
import pandas as pd

# Reading data into a dataframe and removing the first row 
loan_2007 = pd.read_csv('LoanStats3a.csv', skiprows = 1)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- Removing the desc column:
    - which contains a long text explanation for each loan
- Removing the url column:
    - which contains a link to each loan on Lending Club which can only be accessed with an investor account
- Removing all columns containing more than 50% missing values:
    - which allows us to move faster since we can spend less time trying to fill these values

In [53]:
loan_2007 = loan_2007.drop(['url','desc', 'hardship_flag'],axis = 1)
half_row = len(loan)/2
loan_2007 = loan_2007.dropna(thresh = half_row, axis = 1)
loan_2007.head()


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,disbursement_method,debt_settlement_flag
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,...,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,Cash,N


A cleaner file with appropriate columns is available to carry out our feature analysis and predictions. We will us that fie going forward.

In [54]:
loans = pd.read_csv('loans_2007.csv')
loans.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


Wen need to explore the dataset in order to undertsand the features to be used in the prediction process.  We will use the data dictionary to become familiar with what each column represents. We want to pay attention to any features that- 

- leak information from the future (after the loan has already been funded). __This can cause our model to overfit.__
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
- contain redundant information

I have created a spreadsheet that contains the name, data type, first row's value, and description from the data dictionary. This will make analysis of the columns easier. 

In [55]:
data_dict = pd.read_excel('DataDict.xlsx')
data_dict

,name,dtype,description
0,id,object,A unique LC assigned ID for the loan listing.
1,member_id,float64,A unique LC assigned Id for the borrower member.
2,loan_amnt,float64,The listed amount of the loan applied for by t...
3,funded_amnt,float64,The total amount committed to that loan at tha...
4,funded_amnt_inv,float64,The total amount committed by investors for th...
5,term,object,The number of payments on the loan. Values are...
6,int_rate,object,Interest Rate on the loan
7,installment,float64,The monthly payment owed by the borrower if th...
8,grade,object,LC assigned loan grade
9,sub_grade,object,LC assigned loan subgrade


After analyzing each column, we can conclude that the following features need to be removed:

- __id__: randomly generated field by Lending Club for unique identification purposes only (This column is already removed as most of the values were empty)
- __member_id__: also a randomly generated field by Lending Club for unique identification purposes only (This column is already removed as most of the values were empty)
- __funded_amnt__: leaks data from the future (after the loan is already started to be funded)
- __funded_amnt_inv__: also leaks data from the future (after the loan is already started to be funded)
- __grade__: contains redundant information as the interest rate column (int_rate)
- __sub_grade__: also contains redundant information as the interest rate column (int_rate)
- __emp_title__: requires other data and a lot of processing to potentially be useful
- __issue_d__: leaks data from the future (after the loan is already completed funded)

    - Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the int_rate column contains continuous values, which are better suited for machine learning.

- __zip_code__: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- __out_prncp__: leaks data from the future, (after the loan already started to be paid off)
- __out_prncp_inv__: also leaks data from the future, (after the loan already started to be paid off)
- __total_pymnt__: also leaks data from the future, (after the loan already started to be paid off)
- __total_pymnt_inv__: also leaks data from the future, (after the loan already started to be paid off)
- __total_rec_prncp__: also leaks data from the future, (after the loan already started to be paid off)

    - The out_prncp and out_prncp_inv both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the total_pymnt column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.
    
- __total_rec_int__: leaks data from the future, (after the loan already started to be paid off),
- __total_rec_late_fee__: also leaks data from the future, (after the loan already started to be paid off),
- __recoveries__: also leaks data from the future, (after the loan already started to be paid off),
- __collection_recovery_fee__: also leaks data from the future, (after the loan already started to be paid off),
- __last_pymnt_d__: also leaks data from the future, (after the loan already started to be paid off),
- __last_pymnt_amnt__: also leaks data from the future, (after the loan already started to be paid off).


In [56]:
loans = loans.drop(["id", "member_id", "funded_amnt", 
                  "funded_amnt_inv", "grade", "sub_grade", 
                  "emp_title", "issue_d", "zip_code", "out_prncp", 
                  "out_prncp_inv", "total_pymnt", "total_pymnt_inv", 
                  "total_rec_prncp", "total_rec_int", "total_rec_late_fee", 
                  "recoveries", "collection_recovery_fee", "last_pymnt_d", 
                  "last_pymnt_amnt"], axis=1)

print('Total Number of Columns: ',loan.shape[1])


Total Number of Columns:  32


We could reduce the number of columns in the dataset from 54 to 32. 

We need to decide on a target column that we want to use for modeling.

We shoul use the __loan_status__ column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. 
- Currently, this column contains text values and we need to convert it to a numerical one for training a model. 

In [57]:
# Explore the different values for loan_status column
status = loans['loan_status'].value_counts()
status

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. 
- Only the __Fully Paid__ and __Chargeg Off__ values describe the final outcome of the loan.  
- While the __Default__ status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance.

Since we are considering only two values to be able to predict from, we can treat our problem as a binary classification.
- We will reremove all the rows with a status other than Fully Paid and Charged Off.
- Transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case. 


In [58]:
# Dropping rows where status is not 'Fully Paid','Charged Off'
loans = loans.drop(loans[~loans['loan_status'].isin (['Fully Paid','Charged Off' ])].index)

# Transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case. 
status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}
loans = loans.replace(status_replace)

In [60]:
loans.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,...,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,...,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,...,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [61]:
'''
Find out columns that contain only one unique value and remove them. 
These columns won't be useful for the model since they don't add any information to each loan application.
'''

def unique_count(data):
    drop_columns = []
    columns = data.columns
    for each in columns:
        col = data[each].dropna()
        count = len(col.unique())
        if count == 1:
            drop_columns.append(each)
    return drop_columns

drop_cols = unique_count(loans)
drop_cols

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [62]:
# Dropping columns with one unique value

loans = loans.drop(drop_cols, axis = 1)
loans.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'last_credit_pull_d',
       'pub_rec_bankruptcies'],
      dtype='object')

### Feature Engineering - Preparing the Features

For our data to be ready for ML algorithms, we will - 
- Handle missing values
- Convert categorical variables to numerical values
- Remove other variables which are not useful for making predictions.


In [63]:
# Counting number of missing values in the data frame

null_counts = loans.isnull().sum()
null_counts

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

We will delete the column where there are more than 1% of missing values. In addition, we'll remove the remaining rows containing null values.
- pub_rec_bankruptcies has more than 1% missing values.
- Drop rows with null values.


In [74]:
# Dropping the column pub_rec_bankruptcies
loans = loans.drop('pub_rec_bankruptcies', axis = 1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


In [79]:
# Selecting  the columns of object type from loans
object_columns_df = loans.select_dtypes('object')
object_columns_df[:1]

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016


In [90]:
# Counting the number of unique values of object data elements

cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
AK      76
WY      76
SD      60
VT  

 We should analyze the unique value counts for the __purpose__ and __title__ columns to understand which column we want to keep.

In [95]:
purpose_unique_counts = loans["purpose"].value_counts()
title_unique_counts = loans["title"].value_counts()

print(purpose_unique_counts.shape[0])
print(title_unique_counts[0])


14
2068


In [96]:
purpose_unique_counts.head()

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
Name: purpose, dtype: int64

In [97]:
title_unique_counts.head()

Debt Consolidation         2068
Debt Consolidation Loan    1599
Personal Loan               624
Consolidation               488
debt consolidation          466
Name: title, dtype: int64

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. 
- In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).
